# K Nearest Neighbors Model with Bayes

Refined kNN classification model with Bayesian input from the very-accurate breed classification

# Imports

In [20]:
## Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from seaborn import set_style
import csv
set_style("whitegrid")

## More specific imports for this notebook
import joblib ## For saving trained models
from sklearn.neighbors import KNeighborsClassifier ## Import the model here
from sklearn.model_selection import train_test_split ## Import train_test_split
from sklearn.metrics import confusion_matrix ## Import confusion_matrix
from sklearn.metrics import accuracy_score

# Initial Settings and Load Data

In [21]:
X_orig = np.load('../../data/processed_data/specgram_db_pad_trunc.npy')

X_specaug1 = np.load('../../data/processed_data/augment_data/spec_pt_ftmask1.npy')
X_tsh1 = np.load('../../data/processed_data/augment_data/spec_pt_timeshift1.npy')
X_randn1 = np.load('../../data/processed_data/augment_data/spec_pt_randnoise1.npy')
X_tstr1 = np.load('../../data/processed_data/augment_data/spec_pt_timestretch1.npy')

X_specaug2 = np.load('../../data/processed_data/augment_data/spec_pt_ftmask2.npy')
X_tsh2 = np.load('../../data/processed_data/augment_data/spec_pt_timeshift2.npy')
X_randn2 = np.load('../../data/processed_data/augment_data/spec_pt_randnoise2.npy')
X_tstr2 = np.load('../../data/processed_data/augment_data/spec_pt_timestretch2.npy')

X_specaug3 = np.load('../../data/processed_data/augment_data/spec_pt_ftmask3.npy')
X_tsh3 = np.load('../../data/processed_data/augment_data/spec_pt_timeshift3.npy')
X_randn3 = np.load('../../data/processed_data/augment_data/spec_pt_randnoise3.npy')
X_tstr3 = np.load('../../data/processed_data/augment_data/spec_pt_timestretch3.npy')

In [22]:
## Choose what to add to the orignal data
x_data = np.concatenate((X_orig,X_specaug1), axis=0)
X = []
for x in x_data:
    X.append(x.flatten())
aug_file = 'specaug1' #Update when we want to save a file!!!

df = pd.read_csv("../../data/processed_data/metadata.csv")

## Choose which category we are dealing with
situation_to_number = {'brushing': 0, 'food': 1, 'isolation': 2}
sex_to_number = {0:0, 1:1} ## Sex already transfered to number; this is for uniformity of the code
breed_to_number = {'european_shorthair': 0, 'maine_coon': 1}

## Create a new column with numerical values based on the situation mapping
model_types = ['situation', 'sex', 'breed'] 
model_type = model_types[0] + '_' + model_types[1] + '_' + model_types[2]
situation_sex_type = model_types[0] + '_given_' + model_types[1]
situation_breed_type = model_types[0] + '_given_' + model_types[2]

df['numerical_'+model_types[0]] = df[model_types[0]].map(situation_to_number)
df['numerical_'+model_types[1]] = df[model_types[1]].map(sex_to_number)
df['numerical_'+model_types[2]] = df[model_types[2]].map(breed_to_number)
df['numerical_'+situation_sex_type] = 2*df['numerical_'+model_types[0]] + df['numerical_'+model_types[1]]
df['numerical_'+situation_breed_type] = 2*df['numerical_'+model_types[0]] + df['numerical_'+model_types[2]]
df['numerical_'+model_type] = 4*df['numerical_'+model_types[0]] + 2*df['numerical_'+model_types[1]] + df['numerical_'+model_types[2]]

y_situation = np.concatenate((df['numerical_'+model_types[0]].values, df['numerical_'+model_types[0]].values), axis=0)
y_sex = np.concatenate((df['numerical_'+model_types[1]].values, df['numerical_'+model_types[1]].values), axis=0)
y_breed = np.concatenate((df['numerical_'+model_types[2]].values, df['numerical_'+model_types[2]].values), axis=0)
y_situation_sex = np.concatenate((df['numerical_'+situation_sex_type].values, df['numerical_'+situation_sex_type].values), axis=0)
y_situation_breed = np.concatenate((df['numerical_'+situation_breed_type].values, df['numerical_'+situation_breed_type].values), axis=0)

y_orig = df['numerical_'+model_type].values
y = np.concatenate((y_orig, y_orig), axis=0)

X_sit_sex_split = [[],[]]
y_sit_sex_split = [[],[]]
X_sit_breed_split = [[],[]]
y_sit_breed_split = [[],[]]
for a in range(0,len(X)):
    X_sit_sex_split[ y_situation_sex[a]%2 ].append(X[a])
    y_sit_sex_split[ y_situation_sex[a]%2 ].append(y[a])
    X_sit_breed_split[ y_situation_breed[a]%2 ].append(X[a])
    y_sit_breed_split[ y_situation_breed[a]%2 ].append(y[a])
    


print(x_data.shape)
print(y_orig.shape)
print(y_situation.shape)
print(y_sex.shape)
print(y_breed.shape)
print(y_situation_sex.shape)
print(y_situation_breed.shape)
print(len(y_sit_sex_split[0]), len(y_sit_sex_split[1]))
print(len(y_sit_breed_split[0]), len(y_sit_breed_split[1]))
print(y.shape)

(880, 128, 67)
(440,)
(880,)
(880,)
(880,)
(880,)
(880,)
690 190
504 376
(880,)


# Train Test Splits

In [23]:
## Set up the train test splits we need

#Use these variables to automate saving runs with different filesnames
test_size = 1/5
random_state = 440

## Train test split for all three simultaneously
x_train_all, x_test_all, y_train_all, y_test_all = train_test_split(X.copy(), y,
                                        shuffle = True,
                                        random_state = random_state,
                                        test_size = test_size)

## Train test split for modelinlg the situation 
x_train_sit, x_test_sit, y_train_sit, y_test_sit = train_test_split(X.copy(), y_situation,
                                        shuffle = True,
                                        random_state = random_state,
                                        test_size = test_size)

## Train test split for modelinlg the sex
x_train_sex, x_test_sex, y_train_sex, y_test_sex = train_test_split(X.copy(), y_sex,
                                        shuffle = True,
                                        random_state = random_state,
                                        test_size = test_size)

## Train test split for modelinlg the breed
x_train_breed, x_test_breed, y_train_breed, y_test_breed = train_test_split(X.copy(), y_breed,
                                        shuffle = True,
                                        random_state = random_state,
                                        test_size = test_size)

## Train test split for modelinlg the combined situation-sex combination
x_train_sit_sex, x_test_sit_sex, y_train_sit_sex, y_test_sit_sex = train_test_split(X.copy(), y_situation_sex,
                                        shuffle = True,
                                        random_state = random_state,
                                        test_size = test_size)

## Train test split for modelinlg the combined situation-breed combination
x_train_sit_breed, x_test_sit_breed, y_train_sit_breed, y_test_sit_breed = train_test_split(X.copy(), y_situation_breed,
                                        shuffle = True,
                                        random_state = random_state,
                                        test_size = test_size)

## Train test split to modeling the situation for males
x_train_males, x_test_males, y_train_males, y_test_males = train_test_split(X_sit_sex_split[0].copy(), y_sit_sex_split[0],
                                        shuffle = True,
                                        random_state = random_state,
                                        test_size = test_size)

## Train test split for modeling the situation for females
x_train_females, x_test_females, y_train_females, y_test_females = train_test_split(X_sit_sex_split[1].copy(), y_sit_sex_split[1],
                                        shuffle = True,
                                        random_state = random_state,
                                        test_size = test_size)

## Train test split to modeling the situation for shorthairs
x_train_sh, x_test_sh, y_train_sh, y_test_sh = train_test_split(X_sit_breed_split[0].copy(), y_sit_breed_split[0],
                                        shuffle = True,
                                        random_state = random_state,
                                        test_size = test_size)

## Train test split for modeling the situation for maine coons
x_train_mc, x_test_mc, y_train_mc, y_test_mc = train_test_split(X_sit_breed_split[1].copy(), y_sit_breed_split[1],
                                        shuffle = True,
                                        random_state = random_state,
                                        test_size = test_size)

# Fit Models

In [53]:
## Use these variables later to automate saving runs with different filesnames

k = 5
k_all = k
k_situation = k #optimal value 5 from other file
k_sex = k
k_breed = k #optimal value 4 from other file
k_sit_sex = k
k_sit_breed = k
k_males = k
k_females = k
k_shorthair = k 
k_mainecoon = k

## Make the model objects
knn_all = KNeighborsClassifier(k_all)
knn_situation = KNeighborsClassifier(k_situation)
knn_sex = KNeighborsClassifier(k_sex)
knn_breed = KNeighborsClassifier(k_breed)
knn_sit_sex = KNeighborsClassifier(k_sit_sex)
knn_sit_breed = KNeighborsClassifier(k_sit_breed)
knn_males = KNeighborsClassifier(k_males)
knn_females = KNeighborsClassifier(k_females)
knn_shorthair = KNeighborsClassifier(k_shorthair)
knn_mainecoon = KNeighborsClassifier(k_mainecoon)

## Fit the model object
knn_all.fit(x_train_all, y_train_all)
knn_situation.fit(x_train_sit, y_train_sit)
knn_sex.fit(x_train_sex, y_train_sex)
knn_breed.fit(x_train_breed, y_train_breed)
knn_sit_sex.fit(x_train_sit_sex, y_train_sit_sex)
knn_sit_breed.fit(x_train_sit_breed, y_train_sit_breed)
knn_males = knn_males.fit(x_train_males, y_train_males)
knn_females = knn_females.fit(x_train_females, y_train_females)
knn_shorthair.fit(x_train_sh, y_train_sh)
knn_mainecoon.fit(x_train_mc, y_train_mc)

KNeighborsClassifier()

# Assess Model Performances

In [54]:
## Predict on the training sets
y_test_pred_all = knn_all.predict(x_test_all)
y_test_pred_sit = knn_situation.predict(x_test_sit)
y_test_pred_sex = knn_sex.predict(x_test_sex)
y_test_pred_breed = knn_breed.predict(x_test_breed)
y_test_pred_sit_sex = knn_sit_sex.predict(x_test_sit_sex)
y_test_pred_sit_breed = knn_sit_breed.predict(x_test_sit_breed)
y_test_pred_males = knn_males.predict(x_test_males)
y_test_pred_females = knn_females.predict(x_test_females)
y_test_pred_sh = knn_shorthair.predict(x_test_sh)
y_test_pred_mc = knn_mainecoon.predict(x_test_mc)

## Compute confusion matrix for each model
conf_mat_all = confusion_matrix(y_test_all, y_test_pred_all)
conf_mat_situation = confusion_matrix(y_test_sit, y_test_pred_sit)
conf_mat_sex = confusion_matrix(y_test_sex, y_test_pred_sex)
conf_mat_breed = confusion_matrix(y_test_breed, y_test_pred_breed)
conf_mat_sit_sex = confusion_matrix(y_test_sit_sex, y_test_pred_sit_sex)
conf_mat_sit_breed = confusion_matrix(y_test_sit_breed, y_test_pred_sit_breed)
conf_mat_males = confusion_matrix(y_test_males, y_test_pred_males)
conf_mat_females = confusion_matrix(y_test_females, y_test_pred_females)
conf_mat_shorthair = confusion_matrix(y_test_sh, y_test_pred_sh)
conf_mat_mainecoon = confusion_matrix(y_test_mc, y_test_pred_mc)

## Compute accuracy for the model
acc_all = accuracy_score(y_test_all, y_test_pred_all)
acc_situation = accuracy_score(y_test_sit, y_test_pred_sit)
acc_sex = accuracy_score(y_test_sex, y_test_pred_sex)
acc_breed = accuracy_score(y_test_breed, y_test_pred_breed)
acc_sit_sex = accuracy_score(y_test_sit_sex, y_test_pred_sit_sex)
acc_sit_breed = accuracy_score(y_test_sit_breed, y_test_pred_sit_breed)
acc_males = accuracy_score(y_test_males, y_test_pred_males)
acc_females = accuracy_score(y_test_females, y_test_pred_females)
acc_shorthair = accuracy_score(y_test_sh, y_test_pred_sh)
acc_mainecoon = accuracy_score(y_test_mc, y_test_pred_mc)

## Print out collective data
print('k =',k,'used for these kNN models')
print()
print()
print('All three category accuracy:', acc_all)
print()
print('Situation accuracy:', acc_situation)
print()
print('Sex accuracy:', acc_sex)
print()
print('Breed accuracy:', acc_breed)
print()
print('Independence product:', acc_situation * acc_sex * acc_breed)
print()
print()
print('(Situation | Male) accuracy:', acc_males)
print()
print('(Situation | Female) accuracy:', acc_females)
print()
print('(Situation | Sex) accuracy:', acc_sit_sex)
print()
print('(Situation | European Shorthair) accuracy:', acc_shorthair)
print()
print('(Situation | Maine Coon) accuracy:', acc_mainecoon)
print()
print('(Situation | Breed) accuracy:', acc_sit_breed)
print()
print()
print('----- Confusion matrices -----')
print()
print()
print('All three categories confusion matrix:')
print()
print(conf_mat_all)
print()
print()
print('Situation confusion matrix:')
print()
print(conf_mat_situation)
print()
print()
print('Sex confusion matrix:')
print()
print(conf_mat_sex)
print()
print()
print('Breed confusion matrix:')
print()
print(conf_mat_breed)
print()
print()
print('(Situation | Male) confusion matrix:')
print()
print(conf_mat_males)
print()
print()
print('(Situation | Female) confusion matrix:')
print()
print(conf_mat_females)
print()
print()
print('(Situation | Sex) confusion matrix:')
print()
print(conf_mat_sit_sex)
print()
print()
print('(Situation | European Shorthair) confusion matrix:')
print()
print(conf_mat_shorthair)
print()
print()
print('(Situation | Maine Coon) confusion matrix:')
print()
print(conf_mat_mainecoon)
print()
print()
print('(Situation | Breed) confusion matrix:')
print()
print(conf_mat_sit_breed)
print()
print('----------------------------------------------------------------------------------------------------')

k = 5 used for these kNN models


All three category accuracy: 0.7159090909090909

Situation accuracy: 0.7443181818181818

Sex accuracy: 0.8863636363636364

Breed accuracy: 0.9886363636363636

Independence product: 0.6522395637678438


(Situation | Male) accuracy: 0.7246376811594203

(Situation | Female) accuracy: 0.7368421052631579

(Situation | Sex) accuracy: 0.7102272727272727

(Situation | European Shorthair) accuracy: 0.7029702970297029

(Situation | Maine Coon) accuracy: 0.6578947368421053

(Situation | Breed) accuracy: 0.75


----- Confusion matrices -----


All three categories confusion matrix:

[[18  0  1  0  2  1  0  0  5  0  0  0]
 [ 0 10  0  0  0  1  0  2  0  3  0  2]
 [ 0  0  0  0  0  0  0  0  1  0  0  0]
 [ 0  1  0  6  0  0  0  1  0  0  0  0]
 [ 1  0  0  1 11  0  0  0  2  0  0  0]
 [ 0  0  0  0  0  4  0  1  0  1  0  0]
 [ 0  0  1  0  0  1  6  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  2  0  0  0  0]
 [ 4  0  2  0  1  0  0  0 35  0  1  1]
 [ 0  1  0  0  0  3  0  2  0 27  0  1

# A couple data points for X_specaug1

----------------------------------------------------------------------------------------------------

k = 1 used for these kNN models


All three category accuracy: 0.9204545454545454

Situation accuracy: 0.9375

Sex accuracy: 0.9488636363636364

Breed accuracy: 1.0

Independence product: 0.8895596590909091


(Situation | Male) accuracy: 0.8768115942028986

(Situation | Female) accuracy: 0.8947368421052632

(Situation | Sex) accuracy: 0.9204545454545454

(Situation | European Shorthair) accuracy: 0.9405940594059405

(Situation | Maine Coon) accuracy: 0.9210526315789473

(Situation | Breed) accuracy: 0.9375


----- Confusion matrices -----


All three categories confusion matrix:

[[25  0  1  0  0  0  0  0  1  0  0  0]
 [ 0 15  0  0  0  1  0  0  0  0  0  2]
 [ 0  0  1  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  8  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 15  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  5  0  0  0  1  0  0]
 [ 0  0  0  0  1  0  7  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  4  0  0  0  0]
 [ 0  0  2  0  0  0  0  0 42  0  0  0]
 [ 0  0  0  0  0  2  0  2  0 30  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  4  0]
 [ 0  0  0  0  0  0  0  0  0  1  0  6]]


Situation confusion matrix:

[[50  1  3]
 [ 0 32  1]
 [ 2  4 83]]


Sex confusion matrix:

[[137   7]
 [  2  30]]


Breed confusion matrix:

[[99  0]
 [ 0 77]]


(Situation | Male) confusion matrix:

[[24  0  2  0  2  3]
 [ 0 16  0  0  0  2]
 [ 0  0  9  0  1  0]
 [ 0  3  0  3  0  4]
 [ 0  0  0  0 50  0]
 [ 0  0  0  0  0 19]]


(Situation | Female) confusion matrix:

[[1 0 0 0 0 0]
 [0 8 0 2 0 1]
 [0 0 6 0 0 0]
 [0 0 0 5 0 0]
 [0 0 0 0 6 0]
 [0 0 0 1 0 8]]


(Situation | Sex) confusion matrix:

[[40  1  1  0  1  2]
 [ 0  9  0  0  0  0]
 [ 0  0 20  0  1  0]
 [ 0  0  1 11  0  0]
 [ 0  2  2  2 72  0]
 [ 0  0  0  0  1 10]]


(Situation | European Shorthair) confusion matrix:

[[28  0  2  0  1  1]
 [ 0  1  0  0  0  0]
 [ 1  0 12  0  0  0]
 [ 0  0  0  2  0  0]
 [ 0  0  1  0 45  0]
 [ 0  0  0  0  0  7]]


(Situation | Maine Coon) confusion matrix:

[[11  3  1  0  2  0]
 [ 0  3  0  0  0  0]
 [ 0  0 13  0  0  0]
 [ 0  0  0  5  0  0]
 [ 0  0  0  0 25  0]
 [ 0  0  0  0  0 13]]


(Situation | Breed) confusion matrix:

[[27  0  0  0  1  0]
 [ 0 23  0  1  0  2]
 [ 0  0 23  0  0  0]
 [ 0  0  0  9  0  1]
 [ 2  0  0  0 46  0]
 [ 0  0  0  4  0 37]]
 
----------------------------------------------------------------------------------------------------

k = 2 used for these kNN models


All three category accuracy: 0.7840909090909091

Situation accuracy: 0.8125

Sex accuracy: 0.9204545454545454

Breed accuracy: 0.9829545454545454

Independence product: 0.7351215457128099


(Situation | Male) accuracy: 0.8768115942028986

(Situation | Female) accuracy: 0.8421052631578947

(Situation | Sex) accuracy: 0.7840909090909091

(Situation | European Shorthair) accuracy: 0.8217821782178217

(Situation | Maine Coon) accuracy: 0.7763157894736842

(Situation | Breed) accuracy: 0.8068181818181818


----- Confusion matrices -----


All three categories confusion matrix:

[[25  0  2  0  0  0  0  0  0  0  0  0]
 [ 0 16  0  0  0  0  0  0  0  0  0  2]
 [ 0  0  1  0  0  0  0  0  0  0  0  0]
 [ 0  1  0  7  0  0  0  0  0  0  0  0]
 [ 7  0  0  1  7  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  6  0  0  0  0  0  0]
 [ 1  0  1  0  1  0  5  0  0  0  0  0]
 [ 1  0  0  0  1  0  0  2  0  0  0  0]
 [ 5  0  3  0  2  0  0  0 34  0  0  0]
 [ 0  1  0  1  0  2  0  2  0 28  0  0]
 [ 0  0  1  0  0  0  1  0  0  0  2  0]
 [ 0  0  0  0  1  1  0  0  0  0  0  5]]


Situation confusion matrix:

[[52  0  2]
 [11 22  0]
 [11  9 69]]


Sex confusion matrix:

[[141   3]
 [ 11  21]]


Breed confusion matrix:

[[99  0]
 [ 3 74]]


(Situation | Male) confusion matrix:

[[29  1  0  0  1  0]
 [ 0 16  0  1  0  1]
 [ 1  0  9  0  0  0]
 [ 0  6  0  3  0  1]
 [ 1  1  3  0 45  0]
 [ 0  0  0  0  0 19]]


(Situation | Female) confusion matrix:

[[1 0 0 0 0 0]
 [0 9 0 2 0 0]
 [0 0 6 0 0 0]
 [0 0 1 4 0 0]
 [0 0 1 0 5 0]
 [0 1 0 1 0 7]]


(Situation | Sex) confusion matrix:

[[41  2  0  0  0  2]
 [ 1  8  0  0  0  0]
 [ 7  1 13  0  0  0]
 [ 2  1  2  7  0  0]
 [ 6  4  4  2 62  0]
 [ 0  1  2  1  0  7]]


(Situation | European Shorthair) confusion matrix:

[[29  0  2  0  0  1]
 [ 0  1  0  0  0  0]
 [ 6  0  7  0  0  0]
 [ 0  0  0  2  0  0]
 [ 4  0  1  1 40  0]
 [ 0  0  0  2  1  4]]


(Situation | Maine Coon) confusion matrix:

[[11  4  0  0  2  0]
 [ 2  1  0  0  0  0]
 [ 2  0 11  0  0  0]
 [ 1  0  0  4  0  0]
 [ 0  0  2  0 23  0]
 [ 0  0  1  1  2  9]]


(Situation | Breed) confusion matrix:

[[28  0  0  0  0  0]
 [ 0 24  0  0  0  2]
 [ 9  1 13  0  0  0]
 [ 1  0  1  8  0  0]
 [ 9  0  3  0 36  0]
 [ 0  2  1  5  0 33]]
 
----------------------------------------------------------------------------------------------------

k = 3 used for these kNN models


All three category accuracy: 0.7556818181818182

Situation accuracy: 0.8011363636363636

Sex accuracy: 0.9034090909090909

Breed accuracy: 0.9772727272727273

Independence product: 0.7073049222858753


(Situation | Male) accuracy: 0.782608695652174

(Situation | Female) accuracy: 0.7368421052631579

(Situation | Sex) accuracy: 0.7556818181818182

(Situation | European Shorthair) accuracy: 0.7821782178217822

(Situation | Maine Coon) accuracy: 0.7105263157894737

(Situation | Breed) accuracy: 0.7897727272727273


----- Confusion matrices -----


All three categories confusion matrix:

[[22  0  0  0  2  1  0  0  2  0  0  0]
 [ 0 10  0  0  0  3  0  1  0  2  0  2]
 [ 0  0  0  0  0  0  0  0  1  0  0  0]
 [ 0  1  0  7  0  0  0  0  0  0  0  0]
 [ 3  0  0  0 10  0  0  0  2  0  0  0]
 [ 0  0  0  0  0  5  0  1  0  0  0  0]
 [ 1  0  0  0  1  1  5  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  2  0  0  0  0]
 [ 4  0  2  0  1  0  0  0 37  0  0  0]
 [ 0  0  0  1  0  3  0  2  0 28  0  0]
 [ 0  0  1  0  0  0  1  0  0  0  2  0]
 [ 0  0  0  0  1  1  0  0  0  0  0  5]]


Situation confusion matrix:

[[38  8  8]
 [ 5 24  4]
 [ 5  5 79]]


Sex confusion matrix:

[[135   9]
 [  8  24]]


Breed confusion matrix:

[[98  1]
 [ 3 74]]


(Situation | Male) confusion matrix:

[[20  1  6  1  3  0]
 [ 0 13  0  1  0  4]
 [ 1  0  7  1  1  0]
 [ 1  4  0  2  0  3]
 [ 0  1  2  0 47  0]
 [ 0  0  0  0  0 19]]


(Situation | Female) confusion matrix:

[[1 0 0 0 0 0]
 [0 7 0 2 1 1]
 [0 0 4 0 2 0]
 [0 0 0 4 0 1]
 [0 0 1 0 5 0]
 [0 1 0 1 0 7]]


(Situation | Sex) confusion matrix:

[[30  0  7  1  5  2]
 [ 1  7  0  0  1  0]
 [ 3  0 15  1  2  0]
 [ 3  0  2  7  0  0]
 [ 4  2  4  2 66  0]
 [ 0  0  2  1  0  8]]


(Situation | European Shorthair) confusion matrix:

[[23  0  3  0  4  2]
 [ 0  0  0  0  1  0]
 [ 3  0  7  1  2  0]
 [ 0  0  0  2  0  0]
 [ 3  0  1  1 41  0]
 [ 0  0  0  0  1  6]]


(Situation | Maine Coon) confusion matrix:

[[10  4  0  1  2  0]
 [ 2  1  0  0  0  0]
 [ 3  0  6  0  4  0]
 [ 1  0  1  3  0  0]
 [ 0  1  1  0 23  0]
 [ 0  1  1  0  0 11]]


(Situation | Breed) confusion matrix:

[[22  0  2  1  3  0]
 [ 0 18  0  4  0  4]
 [ 4  0 17  0  2  0]
 [ 2  0  0  6  0  2]
 [ 7  0  1  0 40  0]
 [ 0  0  1  4  0 36]]
 
 ----------------------------------------------------------------------------------------------------

k = 4 used for these kNN models


All three category accuracy: 0.6761363636363636

Situation accuracy: 0.7215909090909091

Sex accuracy: 0.9034090909090909

Breed accuracy: 0.9829545454545454

Independence product: 0.6407799953629789


(Situation | Male) accuracy: 0.717391304347826

(Situation | Female) accuracy: 0.7894736842105263

(Situation | Sex) accuracy: 0.6761363636363636

(Situation | European Shorthair) accuracy: 0.7326732673267327

(Situation | Maine Coon) accuracy: 0.6710526315789473

(Situation | Breed) accuracy: 0.7329545454545454


----- Confusion matrices -----


All three categories confusion matrix:

[[18  0  1  0  5  0  0  0  3  0  0  0]
 [ 0  9  0  0  0  3  0  2  0  2  0  2]
 [ 0  0  0  0  0  0  0  0  1  0  0  0]
 [ 0  2  0  5  0  0  0  1  0  0  0  0]
 [ 2  0  0  1  9  0  0  0  3  0  0  0]
 [ 0  0  0  0  0  4  0  1  0  0  0  1]
 [ 2  0  0  0  0  1  5  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  2  0  0  0  0]
 [ 3  0  3  0  2  0  0  0 35  0  1  0]
 [ 0  3  0  0  0  3  0  2  0 25  0  1]
 [ 0  0  1  0  0  1  0  0  0  0  2  0]
 [ 0  0  0  0  1  1  0  0  0  0  0  5]]


Situation confusion matrix:

[[35 10  9]
 [ 6 23  4]
 [10 10 69]]


Sex confusion matrix:

[[139   5]
 [ 12  20]]


Breed confusion matrix:

[[99  0]
 [ 3 74]]


(Situation | Male) confusion matrix:

[[15  1  9  0  6  0]
 [ 0 13  0  0  1  4]
 [ 1  0  7  1  1  0]
 [ 1  3  0  3  0  3]
 [ 1  1  5  0 43  0]
 [ 0  0  0  1  0 18]]


(Situation | Female) confusion matrix:

[[1 0 0 0 0 0]
 [0 6 0 2 1 2]
 [0 0 6 0 0 0]
 [0 0 0 4 0 1]
 [0 0 1 0 5 0]
 [0 0 0 1 0 8]]


(Situation | Sex) confusion matrix:

[[25  1  9  2  6  2]
 [ 2  5  0  1  1  0]
 [ 2  1 13  1  3  1]
 [ 4  0  1  7  0  0]
 [ 6  3  5  2 61  1]
 [ 0  0  3  0  0  8]]


(Situation | European Shorthair) confusion matrix:

[[19  0  4  0  6  3]
 [ 0  0  0  0  1  0]
 [ 3  0  8  0  2  0]
 [ 0  0  0  2  0  0]
 [ 3  0  1  1 41  0]
 [ 0  0  0  2  1  4]]


(Situation | Maine Coon) confusion matrix:

[[ 9  2  1  1  4  0]
 [ 1  1  1  0  0  0]
 [ 2  0  6  0  4  1]
 [ 1  1  1  2  0  0]
 [ 1  0  0  0 24  0]
 [ 0  3  0  0  1  9]]


(Situation | Breed) confusion matrix:

[[19  0  5  0  4  0]
 [ 0 18  0  4  0  4]
 [ 3  1 15  1  3  0]
 [ 2  0  0  7  0  1]
 [ 7  0  2  0 39  0]
 [ 0  3  1  6  0 31]]
 
 ----------------------------------------------------------------------------------------------------

k = 5 used for these kNN models


All three category accuracy: 0.7159090909090909

Situation accuracy: 0.7443181818181818

Sex accuracy: 0.8863636363636364

Breed accuracy: 0.9886363636363636

Independence product: 0.6522395637678438


(Situation | Male) accuracy: 0.7246376811594203

(Situation | Female) accuracy: 0.7368421052631579

(Situation | Sex) accuracy: 0.7102272727272727

(Situation | European Shorthair) accuracy: 0.7029702970297029

(Situation | Maine Coon) accuracy: 0.6578947368421053

(Situation | Breed) accuracy: 0.75


----- Confusion matrices -----


All three categories confusion matrix:

[[18  0  1  0  2  1  0  0  5  0  0  0]
 [ 0 10  0  0  0  1  0  2  0  3  0  2]
 [ 0  0  0  0  0  0  0  0  1  0  0  0]
 [ 0  1  0  6  0  0  0  1  0  0  0  0]
 [ 1  0  0  1 11  0  0  0  2  0  0  0]
 [ 0  0  0  0  0  4  0  1  0  1  0  0]
 [ 0  0  1  0  0  1  6  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  2  0  0  0  0]
 [ 4  0  2  0  1  0  0  0 35  0  1  1]
 [ 0  1  0  0  0  3  0  2  0 27  0  1]
 [ 0  0  1  0  0  1  0  0  0  0  2  0]
 [ 0  0  0  0  0  2  0  0  0  0  0  5]]


Situation confusion matrix:

[[31  9 14]
 [ 5 24  4]
 [ 6  7 76]]


Sex confusion matrix:

[[133  11]
 [  9  23]]


Breed confusion matrix:

[[99  0]
 [ 2 75]]


(Situation | Male) confusion matrix:

[[15  1  9  0  6  0]
 [ 0 10  0  2  1  5]
 [ 1  0  8  1  0  0]
 [ 1  2  0  3  0  4]
 [ 0  0  4  0 46  0]
 [ 0  0  0  1  0 18]]


(Situation | Female) confusion matrix:

[[1 0 0 0 0 0]
 [0 6 0 2 1 2]
 [0 0 5 0 1 0]
 [0 1 1 3 0 0]
 [0 0 0 0 6 0]
 [0 1 0 1 0 7]]


(Situation | Sex) confusion matrix:

[[26  1  6  2  8  2]
 [ 1  6  0  1  1  0]
 [ 1  1 15  1  3  0]
 [ 2  1  1  8  0  0]
 [ 5  2  4  2 63  2]
 [ 0  0  2  1  1  7]]


(Situation | European Shorthair) confusion matrix:

[[16  0  6  0  8  2]
 [ 0  0  0  0  1  0]
 [ 2  0  8  1  2  0]
 [ 0  0  0  2  0  0]
 [ 2  0  2  1 41  0]
 [ 1  0  0  2  0  4]]


(Situation | Maine Coon) confusion matrix:

[[ 8  2  4  1  2  0]
 [ 1  1  1  0  0  0]
 [ 3  0  6  0  3  1]
 [ 1  1  1  2  0  0]
 [ 1  0  0  0 24  0]
 [ 0  3  0  0  1  9]]


(Situation | Breed) confusion matrix:

[[19  0  2  1  6  0]
 [ 0 15  0  4  0  7]
 [ 2  1 18  0  2  0]
 [ 2  0  0  6  0  2]
 [ 7  0  1  1 39  0]
 [ 0  1  0  5  0 35]]
 
 ----------------------------------------------------------------------------------------------------